In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

cam = 2
model_version = "my_model_11.3 - (128x128).keras"

Im_Width = 128
Im_Height = 128

def process_frame(frame, target_size):
    """
    Processes a webcam frame for model prediction.
    
    Args:
        frame (np.array): Captured frame from the webcam.
        target_size (tuple): Target size for resizing the frame (width, height).

    Returns:
        np.array: Preprocessed image ready for model prediction.
    """
    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

def predict_frame(frame, model, target_size):
    """
    Predicts the class of a single frame using the model.

    Args:
        frame (np.array): Captured frame from the webcam.
        model: Loaded Keras model.
        target_size (tuple): Target size for resizing the frame.

    Returns:
        str: Predicted class label.
        float: Confidence score.
    """
    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

    # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        return "Non-Defective", prediction
    else:
        return "Defective", prediction

def run_webcam_with_model(model_path, target_size=(Im_Width, Im_Height)):
    """
    Opens the webcam, captures frames, and processes them using the model.

    Args:
        model_path (str): Path to the saved Keras model.
        target_size (tuple): Target size for resizing frames (width, height).
    """
    # Load the model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    # Open the webcam
    cap = cv2.VideoCapture(cam)
    if not cap.isOpened():
        raise IOError("Cannot open the webcam")

    print("Webcam opened. Press 'ESC' to exit.")
    try:
        while True:
            # Capture frame from the webcam
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame. Exiting...")
                break

            # Process and predict
            feedback, confidence = predict_frame(frame, model, target_size)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            prob = abs(round(confidence,2)- 0.5)*200

            # Display feedback on the frame
            cv2.putText(frame, f"Prediction: {feedback}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Confidence: {prob:.0f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Show the frame in a window
            cv2.imshow("Webcam Feed", frame)

            # Exit on 'ESC' key
            if cv2.waitKey(1) & 0xFF == 27:  # 27 is the ASCII code for ESC
                print("Exiting...")
                break

    finally:
        # Release resources
        cap.release()
        cv2.destroyAllWindows()

# Run the script
if __name__ == "__main__":
    model_path = model_version  # Path to your saved model
    run_webcam_with_model(model_path, target_size=(Im_Width, Im_Height))


Loading model...
Model loaded successfully.
Webcam opened. Press 'ESC' to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 893ms/step
Raw Prediction: 0.12283698469400406
Prediction: Defective (Confidence: 0.12)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Raw Prediction: 0.16348788142204285
Prediction: Defective (Confidence: 0.16)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Raw Prediction: 0.24808397889137268
Prediction: Defective (Confidence: 0.25)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Raw Prediction: 0.18135325610637665
Prediction: Defective (Confidence: 0.18)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Raw Prediction: 0.11243133246898651
Prediction: Defective (Confidence: 0.11)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Raw Prediction: 0.07930479198694229
Prediction: Defective (Confidence: 0.08)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Raw Prediction: 0.14718064665794373
Prediction: Defective (Confidence: 0.15)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Raw Prediction: 0.07942651957273483
Prediction: Defective (Confidence: 0.08)